<p style="align: center;"><img src="https://static.tildacdn.com/tild6636-3531-4239-b465-376364646465/Deep_Learning_School.png" width="400"></p>

# Домашнее задание. Обучение языковой модели с помощью LSTM (10 баллов)

Э
В этом задании Вам предстоит обучить языковую модель с помощью рекуррентной нейронной сети. В отличие от семинарского занятия, Вам необходимо будет работать с отдельными словами, а не буквами.


Установим модуль ```datasets```, чтобы нам проще было работать с данными.

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00


Импорт необходимых библиотек

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from datasets import load_dataset
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
import nltk

from collections import Counter
from typing import List

import seaborn
seaborn.set(palette='summer')

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Подготовка данных

Воспользуемся датасетом imdb. В нем хранятся отзывы о фильмах с сайта imdb. Загрузим данные с помощью функции ```load_dataset```

In [5]:
# Загрузим датасет
dataset = load_dataset('imdb')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

### Препроцессинг данных и создание словаря (1 балл)

Далее вам необходмо самостоятельно произвести препроцессинг данных и получить словарь или же просто ```set``` строк. Что необходимо сделать:

1. Разделить отдельные тренировочные примеры на отдельные предложения с помощью функции ```sent_tokenize``` из бибилиотеки ```nltk```. Каждое отдельное предложение будет одним тренировочным примером.
2. Оставить только те предложения, в которых меньше ```word_threshold``` слов.
3. Посчитать частоту вхождения каждого слова в оставшихся предложениях. Для деления предлоения на отдельные слова удобно использовать функцию ```word_tokenize```.
4. Создать объект ```vocab``` класса ```set```, положить в него служебные токены '\<unk\>', '\<bos\>', '\<eos\>', '\<pad\>' и vocab_size самых частовстречающихся слов.   

In [6]:
class DataSetProcessor:
    def __init__(self, dataset_name, text_column, split, word_threshold, vocab_size, service_tokens=None):
        self.dataset_name = dataset_name
        self.text_column = text_column
        self.split = split
        self.word_threshold = word_threshold
        self.vocab_size = vocab_size
        self.service_tokens = service_tokens or ['<unk>', '<bos>', '<eos>', '<pad>']
        self.dataset = None
        self.sentences = []
        self.vocab = set()

    def load_dataset(self):
        self.dataset = load_dataset(self.dataset_name)

    def preprocess_data(self):
        text_data = self.dataset[self.split][self.text_column]
        self.sentences = [sent for text in text_data for sent in sent_tokenize(text)
                          if len(word_tokenize(sent)) < self.word_threshold]

    def build_vocabulary(self):
        words = [word for sent in self.sentences for word in word_tokenize(sent)]
        word_counts = Counter(words)
        self.vocab = set(self.service_tokens + [word for word, _ in word_counts.most_common(self.vocab_size)])

    def process(self):
        self.load_dataset()
        self.preprocess_data()
        self.build_vocabulary()

    def get_sentences(self):
        return self.sentences

    def get_vocabulary(self):
        return self.vocab

In [7]:
DATASET_NAME = 'imdb'
TEXT_COLUMN = 'text'
TRAIN_SPLIT = 'train'
WORD_THRESHOLD = 32
VOCAB_SIZE = 40000
SERVICE_TOKENS = ['<unk>', '<bos>', '<eos>', '<pad>']


train_dataset_processor = DataSetProcessor(dataset_name=DATASET_NAME, text_column=TEXT_COLUMN, split=TRAIN_SPLIT,
                          word_threshold=WORD_THRESHOLD, vocab_size=VOCAB_SIZE, service_tokens=SERVICE_TOKENS)

train_dataset_processor.process()

sentences = train_dataset_processor.get_sentences()
vocab = train_dataset_processor.get_vocabulary()

assert '<unk>' in vocab
assert '<bos>' in vocab
assert '<eos>' in vocab
assert '<pad>' in vocab
assert len(vocab) == VOCAB_SIZE + 4


print(f"Всего предложений: {len(sentences)}")
print(f"Всего слов в словаре: {len(vocab)}")

Всего предложений: 198801
Всего слов в словаре: 40004


### Подготовка датасета (1 балл)

Далее, как и в семинарском занятии, подготовим датасеты и даталоадеры.

В классе ```WordDataset``` вам необходимо реализовать метод ```__getitem__```, который будет возвращать сэмпл данных по входному idx, то есть список целых чисел (индексов слов).

Внутри этого метода необходимо добавить служебные токены начала и конца последовательности, а также токенизировать соответствующее предложение с помощью ```word_tokenize``` и сопоставить ему индексы из ```word2ind```.

In [8]:
class Vocabulary:
    def __init__(self, vocab):
        self.word2ind = {char: i for i, char in enumerate(vocab)}
        self.ind2word = {i: char for char, i in self.word2ind.items()}


class WordDataset:
    def __init__(self, sentences, vocab):
        self.data = sentences
        self.vocab = vocab
        self.unk_id = vocab.word2ind['<unk>']
        self.bos_id = vocab.word2ind['<bos>']
        self.eos_id = vocab.word2ind['<eos>']
        self.pad_id = vocab.word2ind['<pad>']

    def __getitem__(self, idx: int) -> List[int]:
        sentence = self.data[idx]
        tokenized_sentence = [self.bos_id] + [self.vocab.word2ind.get(word, self.unk_id) for word in word_tokenize(sentence)] + [self.eos_id]
        return tokenized_sentence

    def __len__(self) -> int:
        return len(self.data)


class Collator:
    def __init__(self, pad_id):
        self.pad_id = pad_id

    def __call__(self, input_batch: List[List[int]]) -> torch.Tensor:
        seq_lens = [len(x) for x in input_batch]
        max_seq_len = max(seq_lens)
        new_batch = []
        for sequence in input_batch:
            for _ in range(max_seq_len - len(sequence)):
                sequence.append(self.pad_id)
            new_batch.append(sequence)
        sequences = torch.LongTensor(new_batch).to(device)
        new_batch = {
            'input_ids': sequences[:, :-1],
            'target_ids': sequences[:, 1:]
        }
        return new_batch


class DataSplitter:
    def __init__(self, sentences, test_size, eval_size):
        self.sentences = sentences
        self.test_size = test_size
        self.eval_size = eval_size

    def split_data(self):
        train_sentences, eval_sentences = train_test_split(self.sentences, test_size=self.test_size)
        eval_sentences, test_sentences = train_test_split(eval_sentences, test_size=self.eval_size)
        return train_sentences, eval_sentences, test_sentences


class DataLoaderFactory:
    def __init__(self, vocab, batch_size, device):
        self.vocab = vocab
        self.batch_size = batch_size
        self.device = device

    def create_data_loaders(self, train_sentences, eval_sentences, test_sentences):
        train_dataset = WordDataset(train_sentences, self.vocab)
        eval_dataset = WordDataset(eval_sentences, self.vocab)
        test_dataset = WordDataset(test_sentences, self.vocab)

        collate_fn = Collator(self.vocab.word2ind['<pad>'])

        train_dataloader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=self.batch_size)
        eval_dataloader = DataLoader(eval_dataset, collate_fn=collate_fn, batch_size=self.batch_size)
        test_dataloader = DataLoader(test_dataset, collate_fn=collate_fn, batch_size=self.batch_size)

        return train_dataloader, eval_dataloader, test_dataloader


vocab = Vocabulary(vocab)
data_splitter = DataSplitter(sentences, test_size=0.2, eval_size=0.5)
train_sentences, eval_sentences, test_sentences = data_splitter.split_data()

data_loader_factory = DataLoaderFactory(vocab, batch_size=128, device=device)
train_dataloader, eval_dataloader, test_dataloader = data_loader_factory.create_data_loaders(
    train_sentences, eval_sentences, test_sentences
)

## Обучение и архитектура модели

Вам необходимо на практике проверить, что влияет на качество языковых моделей. В этом задании нужно провести серию экспериментов с различными вариантами языковых моделей и сравнить различия в конечной перплексии на тестовом множестве.

Возмоэные идеи для экспериментов:

* Различные RNN-блоки, например, LSTM или GRU. Также можно добавить сразу несколько RNN блоков друг над другом с помощью аргумента num_layers. Вам поможет официальная документация [здесь](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)
* Различные размеры скрытого состояния. Различное количество линейных слоев после RNN-блока. Различные функции активации.
* Добавление нормализаций в виде Dropout, BatchNorm или LayerNorm
* Различные аргументы для оптимизации, например, подбор оптимального learning rate или тип алгоритма оптимизации SGD, Adam, RMSProp и другие
* Любые другие идеи и подходы

После проведения экспериментов необходимо составить таблицу результатов, в которой описан каждый эксперимент и посчитана перплексия на тестовом множестве.

Учтите, что эксперименты, которые различаются, например, только размером скрытого состояния или количеством линейных слоев считаются, как один эксперимент.

Успехов!

### Train loop (1 балл)

Напишите функцию для обучения модели.

In [9]:
class Trainer:
    def __init__(self, model, criterion, optimizer, device):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.device = device

    def train(self, dataloader, num_epochs):
        self.model.train()
        for epoch in range(num_epochs):
            epoch_loss = 0.0
            epoch_perplexity = 0.0
            progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
            for batch in progress_bar:
                input_batch = batch['input_ids'].to(self.device)
                target_batch = batch['target_ids'].to(self.device)
                self.optimizer.zero_grad()
                logits = self.model(input_batch)
                loss = self.criterion(logits.view(-1, logits.size(-1)), target_batch.flatten())
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item()
                epoch_perplexity += torch.exp(loss).item()
                progress_bar.set_postfix({"Batch Loss": loss.item(), "Batch Perplexity": torch.exp(loss).item()})
            epoch_loss /= len(dataloader)
            epoch_perplexity /= len(dataloader)
            self.model.eval()
            eval_perplexity = self.evaluate(dataloader)
            self.model.train()
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Train Perplexity: {epoch_perplexity:.4f}, Eval Perplexity: {eval_perplexity:.4f}")

    def evaluate(self, dataloader):
        self.model.eval()
        perplexity = []
        with torch.no_grad():
            for batch in dataloader:
                input_batch = batch['input_ids'].to(self.device)
                target_batch = batch['target_ids'].to(self.device)

                logits = self.model(input_batch)
                loss = self.criterion(logits.view(-1, logits.size(-1)), target_batch.flatten())
                perplexity.append(torch.exp(loss).item())

        perplexity = sum(perplexity) / len(perplexity)
        return perplexity


In [10]:
class LanguageModelBase(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

    def forward(self, input_batch: torch.Tensor) -> torch.Tensor:
        raise NotImplementedError("Subclasses must implement the forward method.")

In [11]:
class Experiment:
    def __init__(self, model_factory, criterion, optimizer_factory, device):
        self.model = model_factory().to(device)
        self.criterion = criterion
        self.optimizer = optimizer_factory(self.model.parameters())
        self.device = device

    def run(self, train_dataloader, test_dataloader, num_epochs):
        trainer = Trainer(self.model, self.criterion, self.optimizer, self.device)
        trainer.train(train_dataloader, num_epochs)
        test_perplexity = trainer.evaluate(test_dataloader)
        print(f"Test Perplexity: {test_perplexity:.4f}")

### Первый эксперимент (2 балла)

Определите архитектуру модели и обучите её.

In [12]:
class LSTMLanguageModel(LanguageModelBase):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate):
        super().__init__(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, dropout=dropout_rate, batch_first=True)
        self.linear = nn.Linear(hidden_dim, vocab_size)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, input_batch):
        embeddings = self.embedding(input_batch)
        embeddings = self.dropout(embeddings)
        lstm_output, _ = self.lstm(embeddings)
        lstm_output = self.dropout(lstm_output)
        logits = self.linear(lstm_output)
        return logits



In [13]:
vocab_size = len(vocab.word2ind)
embedding_dim = 128
hidden_dim = 256
num_layers = 2
dropout_rate = 0.5
num_epochs = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Эксперимент с LSTM моделью
lstm_model_factory = lambda: LSTMLanguageModel(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate)
lstm_optimizer_factory = lambda model_params: torch.optim.Adam(model_params)
lstm_criterion = nn.CrossEntropyLoss()
lstm_experiment = Experiment(lstm_model_factory, lstm_criterion, lstm_optimizer_factory, device)
lstm_experiment.run(train_dataloader, eval_dataloader, num_epochs)


Epoch 1/5:   0%|          | 0/1243 [00:00<?, ?batch/s]

Epoch 1/5, Loss: 3.3393, Train Perplexity: 154.8152, Eval Perplexity: 18.7060


Epoch 2/5:   0%|          | 0/1243 [00:00<?, ?batch/s]

Epoch 2/5, Loss: 2.9454, Train Perplexity: 19.1962, Eval Perplexity: 16.0506


Epoch 3/5:   0%|          | 0/1243 [00:00<?, ?batch/s]

Epoch 3/5, Loss: 2.8459, Train Perplexity: 17.3628, Eval Perplexity: 14.7287


Epoch 4/5:   0%|          | 0/1243 [00:00<?, ?batch/s]

Epoch 4/5, Loss: 2.7834, Train Perplexity: 16.3046, Eval Perplexity: 13.8671


Epoch 5/5:   0%|          | 0/1243 [00:00<?, ?batch/s]

Epoch 5/5, Loss: 2.7376, Train Perplexity: 15.5708, Eval Perplexity: 13.2449
Test Perplexity: 14.3202


### Второй эксперимент (2 балла)

Попробуйте что-то поменять в модели или в пайплайне обучения, идеи для экспериментов можно подсмотреть выше.

In [14]:
class GRULanguageModel(LanguageModelBase):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate):
        super().__init__(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers, dropout=dropout_rate)
        self.linear1 = nn.Linear(hidden_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input_batch: torch.Tensor) -> torch.Tensor:
        embeddings = self.embedding(input_batch)
        embeddings = self.dropout(embeddings)
        gru_output, _ = self.gru(embeddings)
        gru_output = self.dropout(gru_output)
        output = self.linear1(gru_output)
        output = self.relu(output)
        output = self.dropout(output)
        logits = self.linear2(output)
        return logits

In [15]:
vocab_size = len(vocab.word2ind)
embedding_dim = 64
hidden_dim = 128
num_layers = 4
dropout_rate = 0.25
num_epochs = 5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Эксперимент с GRU моделью
gru_model_factory = lambda: GRULanguageModel(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate)
gru_optimizer_factory = lambda model_params: torch.optim.Adam(model_params)
gru_criterion = nn.CrossEntropyLoss()
gru_experiment = Experiment(lstm_model_factory, lstm_criterion, gru_optimizer_factory, device)
gru_experiment.run(train_dataloader, test_dataloader, num_epochs)

Epoch 1/5:   0%|          | 0/1243 [00:00<?, ?batch/s]

Epoch 1/5, Loss: 3.8316, Train Perplexity: 308.8187, Eval Perplexity: 29.4413


Epoch 2/5:   0%|          | 0/1243 [00:00<?, ?batch/s]

Epoch 2/5, Loss: 3.2868, Train Perplexity: 27.1042, Eval Perplexity: 22.9158


Epoch 3/5:   0%|          | 0/1243 [00:00<?, ?batch/s]

Epoch 3/5, Loss: 3.0954, Train Perplexity: 22.3222, Eval Perplexity: 19.3764


Epoch 4/5:   0%|          | 0/1243 [00:00<?, ?batch/s]

Epoch 4/5, Loss: 2.9697, Train Perplexity: 19.6628, Eval Perplexity: 17.4837


Epoch 5/5:   0%|          | 0/1243 [00:00<?, ?batch/s]

Epoch 5/5, Loss: 2.8921, Train Perplexity: 18.1853, Eval Perplexity: 16.3072
Test Perplexity: 16.9192


### Отчет (2 балла)

Опишите проведенные эксперименты. Сравните перплексии полученных моделей. Предложите идеи по улучшению качества моделей.

## Отчет по экспериментам с языковыми моделями
В данном эксперименте были обучены и сравнены две модели языкового моделирования на основе рекуррентных нейронных сетей: LSTM (Long Short-Term Memory) и GRU (Gated Recurrent Unit).

###Сравнение перплексий полученных моделей
1. LSTM-модель:
* Перплексия на обучающем наборе данных: 15.5708
* Перплексия на тестовом наборе данных: 14.3202
2. GRU-модель:
* Перплексия на обучающем наборе данных: 18.1853
* Перплексия на тестовом наборе данных: 16.9192

По результатам эксперимента видно, что LSTM-модель достигла меньшей перплексии как на обучающем, так и на тестовом наборах данных по сравнению с GRU-моделью. Это говорит о том, что LSTM-модель лучше справилась с задачей языкового моделирования в данном случае.

### Идеи по улучшению качества моделей
1. **Увеличение размера обучающих данных**: Использование более обширных текстов для обучения моделей может помочь улучшить их обобщающую способность и снизить перплексию.
2. **Настройка гиперпараметров:** Эксперименты с различными значениями гиперпараметров, таких как размерность эмбеддингов, размерность скрытых состояний, количество слоев и уровень дропаута, могут помочь найти оптимальную конфигурацию для каждой модели.
3. **Использование более сложных архитектур:** Рассмотрение других архитектур, таких как двунаправленные рекуррентные сети (Bidirectional RNNs) или механизмы внимания (Attention Mechanisms), может улучшить способность моделей учитывать контекст и зависимости между словами.
4. **Регуляризация:** Применение дополнительных методов регуляризации, таких как L1/L2 регуляризация или техники обрезки градиентов (gradient clipping), может помочь предотвратить переобучение и улучшить обобщающую способность моделей.